In [ ]:
!pip install evaluate datasets
!pip install accelerate -U
!pip install rouge_score

In [2]:
import os
import json

import numpy as np

import warnings
import os

from transformers import (GPT2LMHeadModel,
                          GPT2Tokenizer,
                          TrainingArguments,
                          Trainer,
                          DataCollatorForLanguageModeling,
                          TextDataset,
                          pipeline)

from datasets import load_dataset

import evaluate
from nltk.translate.bleu_score import sentence_bleu

#from rouge_score import rouge_score
from tqdm import tqdm

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: ', device)

warnings.filterwarnings("ignore")

2024-02-29 21:18:21.667053: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 21:18:21.667159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 21:18:21.804175: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Device:  cuda


Была взять сберовская rugpt3-small и датасет horoscope 

In [ ]:
model_name_or_path = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(device)

In [ ]:
dataset_path = "/kaggle/input/horoscopes-ru-hf/"
dataset = load_dataset("json", 
                       data_files={"train": os.path.join(dataset_path, "json_horoscopes_train.json"), 
                                    "validation": os.path.join(dataset_path, "json_horoscopes_test.json")}, 
                       field="data")
dataset

In [6]:
dataset['train'][0]

{'date': '2004-01-01',
 'sign': 'овен',
 'text': 'Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.'}

Запустим инференс на претрейн модельке

In [ ]:
def preprocess_function(sample):
    return tokenizer(sample["text"])

tokenized_ds = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

In [8]:
training_args = TrainingArguments(
    output_dir = "./gptru_small_1000stpes_default",
    overwrite_output_dir = True,
    max_steps = 1000,
    fp16 = True,
    logging_first_step=True,
    logging_steps=50,
    per_device_train_batch_size = 8,
    save_strategy = 'epoch',
    report_to='tensorboard'
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    data_collator=data_collator,
)

In [10]:
trainer.train()

Step,Training Loss
1,5.983100
50,3.607300
100,3.338000
150,3.288200
200,3.245800
250,3.212100
300,3.181400
350,3.163000
400,3.100500
450,3.129100


TrainOutput(global_step=1000, training_loss=3.1281999945640564, metrics={'train_runtime': 180.2057, 'train_samples_per_second': 44.394, 'train_steps_per_second': 5.549, 'total_flos': 485545918464000.0, 'train_loss': 3.1281999945640564, 'epoch': 0.12})

In [11]:
nlls = []
for item in tqdm(tokenized_ds['validation']): 
    with torch.no_grad():
        outputs = model(input_ids=torch.tensor(item['input_ids']).to(device),
                        attention_mask=torch.tensor(item['attention_mask']).to(device),
                        labels=torch.tensor(item['input_ids']).to(device))
        neg_log_likelihood = outputs.loss.to('cpu')

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).mean())
print('Perplexity:', ppl.item())

100%|██████████| 6976/6976 [01:52<00:00, 62.19it/s]


Perplexity: 20.01951789855957


In [24]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=40)
prompt = "Завтра у Рыб "
print(generator(prompt)[0]['generated_text'])

Завтра у Рыб  есть шанс получить хороший совет, но  он не всегда будет  полным. В это время  вы можете  столкнуться с  необходимостью  пересмотреть свои планы,  пересмотреть  свои


In [13]:
training_args = TrainingArguments(
    output_dir = "./gptru_small_1000steps_warmup_cosine_lr2e-5",
    overwrite_output_dir = True,
    max_steps = 1000,
    warmup_steps = 100,
    logging_first_step=True,
    logging_steps=50,
    fp16 = True,
    per_device_train_batch_size = 8,
    learning_rate = 2e-5,
    save_strategy = 'epoch',
    lr_scheduler_type = 'cosine',
    report_to='tensorboard'
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    data_collator=data_collator,
)

In [15]:
trainer.train()

Step,Training Loss
1,2.870600
50,2.567400
100,2.512700
150,2.538100
200,2.541500
250,2.533800
300,2.551000
350,2.582200
400,2.546400
450,2.620900


TrainOutput(global_step=1000, training_loss=2.671498342752457, metrics={'train_runtime': 179.7193, 'train_samples_per_second': 44.514, 'train_steps_per_second': 5.564, 'total_flos': 485545918464000.0, 'train_loss': 2.671498342752457, 'epoch': 0.12})

In [16]:
nlls = []
for item in tqdm(tokenized_ds['validation']): 
    with torch.no_grad():
        outputs = model(input_ids=torch.tensor(item['input_ids']).to(device),
                        attention_mask=torch.tensor(item['attention_mask']).to(device),
                        labels=torch.tensor(item['input_ids']).to(device))
        neg_log_likelihood = outputs.loss.to('cpu')

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).mean())
print('Perplexity:', ppl.item())

100%|██████████| 6976/6976 [01:52<00:00, 61.87it/s]


Perplexity: 19.664276123046875


In [26]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=40)
prompt = "Завтра у Рыб "
print(generator(prompt)[0]['generated_text'])

Завтра у Рыб  может появиться новый план действий. Не исключено, что вы будете действовать в рамках своего собственного плана, но не забывайте, что он не всегда будет выполним. В конце дня вы


Подбором параметров удалось чуть чуть улучшить значение perplexity на тестовой выборке